<a href="https://colab.research.google.com/github/chi-saku/comfyui-google-colab-starter/blob/main/Forge_Complete_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ================================================================
# Stable Diffusion WebUI Forge - 最終版（初期セットアップ）
# 生成画像は Google Drive: MyDrive/AI/outputs/forge に自動保存
# ================================================================
import os, sys, subprocess, shutil
from pathlib import Path

def safe_run(cmd, timeout=180, **kwargs):
    """タイムアウト付き安全実行"""
    try:
        return subprocess.run(cmd, timeout=timeout, text=True, **kwargs)
    except subprocess.TimeoutExpired:
        print(f"⚠️ タイムアウト: {' '.join(cmd) if isinstance(cmd, list) else cmd}")
        return None

print("🚀 Stable Diffusion WebUI Forge - 最終版セットアップ開始...")

# 1) Google Driveマウント（エラー回復機能付き）
print("1️⃣ Google Driveマウント中...")
from google.colab import drive
try:
    drive.mount('/content/drive')
except Exception:
    try: drive.flush_and_unmount()
    except: pass
    os.system('fusermount -u /content/drive 2>/dev/null || true')
    os.makedirs('/content/drive', exist_ok=True)
    drive.mount('/content/drive', force_remount=True)

# 2) 基本パス設定
WEBUI_DIR = Path('/content/stable-diffusion-webui-forge')
AI_BASE = Path('/content/drive/MyDrive/AI')
MODELS_BASE = AI_BASE / 'models'
OUTPUTS_BASE = AI_BASE / 'outputs' / 'forge'

# 3) Forgeリポジトリ取得
print("2️⃣ Forgeリポジトリ取得中...")
if WEBUI_DIR.exists():
    shutil.rmtree(WEBUI_DIR, ignore_errors=True)
safe_run(['git', 'clone', '--depth', '1',
          'https://github.com/lllyasviel/stable-diffusion-webui-forge',
          str(WEBUI_DIR)], check=True)

# 4) 2025年環境対応（PyTorch 2.8+ 対応）
print("3️⃣ 2025年環境最適化中...")
safe_run([sys.executable, '-m', 'pip', 'uninstall', '-y', 'xformers'])

# 5) 重要依存関係インストール（タイムアウト対策）
print("4️⃣ 重要依存関係インストール中...")
critical_deps = ['gradio>=4.0.0', 'fastapi>=0.110.0', 'uvicorn>=0.23.0']
for dep in critical_deps:
    print(f"   📦 {dep}")
    safe_run([sys.executable, '-m', 'pip', 'install', '--upgrade', dep])

# 6) モデルフォルダ自動リンク（大文字小文字対応）
print("5️⃣ モデルフォルダリンク中...")
def find_model_folder(base_path, target_names):
    """大文字小文字を考慮してフォルダを検索"""
    if not base_path.exists():
        return None
    target_lower = [name.lower() for name in target_names]
    for folder in base_path.iterdir():
        if folder.is_dir() and folder.name.lower() in target_lower:
            return folder
    return None

def safe_relink(forge_path, real_path):
    """安全なシンボリックリンク作成"""
    if not real_path or not real_path.exists():
        return False

    # 既存の正しいリンクはスキップ（効率化）
    if forge_path.is_symlink():
        try:
            if Path(os.readlink(forge_path)).resolve() == real_path.resolve():
                return True
        except: pass

    # 既存ファイル/フォルダを安全に処理
    if forge_path.exists() or forge_path.is_symlink():
        if forge_path.is_symlink():
            forge_path.unlink()
        elif forge_path.is_dir():
            if any(forge_path.iterdir()):
                backup = forge_path.with_suffix('.backup')
                shutil.move(str(forge_path), str(backup))
                print(f"  📦 退避: {forge_path.name} → {backup.name}")
            else:
                shutil.rmtree(forge_path, ignore_errors=True)
        else:
            forge_path.unlink()

    # 新しいリンク作成
    try:
        forge_path.parent.mkdir(parents=True, exist_ok=True)
        os.symlink(real_path, forge_path, target_is_directory=True)
        return True
    except Exception as e:
        print(f"  ❌ リンクエラー {forge_path.name}: {e}")
        return False

# モデルタイプマッピング
model_mappings = {
    'Stable-diffusion': ['Stable-diffusion', 'stable-diffusion', 'checkpoints'],
    'Lora': ['Lora', 'lora', 'LoRA'],
    'VAE': ['VAE', 'vae', 'Vae'],
    'ControlNet': ['ControlNet', 'controlnet'],
    'embeddings': ['embeddings', 'Embeddings'],
    'hypernetworks': ['hypernetworks', 'Hypernetworks']
}

linked_count = 0
for forge_name, search_names in model_mappings.items():
    real_folder = find_model_folder(MODELS_BASE, search_names)
    forge_path = WEBUI_DIR / 'models' / forge_name

    if real_folder:
        if safe_relink(forge_path, real_folder):
            model_count = len(list(real_folder.glob('*.safetensors')) + list(real_folder.glob('*.ckpt')))
            print(f"  ✅ {forge_name} → {real_folder.name} ({model_count}個)")
            linked_count += 1
    else:
        print(f"  ⚠️ {forge_name}: フォルダなし")

print(f"6️⃣ モデルリンク完了: {linked_count}/{len(model_mappings)}個")

# 7) 🎯【重要】画像出力をGoogle Driveに自動保存設定
print("7️⃣ 画像出力をGoogle Driveに設定中...")
OUTPUTS_BASE.mkdir(parents=True, exist_ok=True)
forge_outputs = WEBUI_DIR / 'outputs'

# 既存outputsを安全に処理
if forge_outputs.exists() or forge_outputs.is_symlink():
    if forge_outputs.is_symlink():
        forge_outputs.unlink()
    elif forge_outputs.is_dir():
        if any(forge_outputs.iterdir()):
            backup = forge_outputs.with_suffix('.backup')
            shutil.move(str(forge_outputs), str(backup))
            print(f"  📦 既存画像退避: {backup.name}")
        else:
            shutil.rmtree(forge_outputs, ignore_errors=True)

# Google DriveへのoutputsリンクPython作成
try:
    os.symlink(OUTPUTS_BASE, forge_outputs, target_is_directory=True)
    print(f"✅ 画像自動保存設定完了: outputs → {OUTPUTS_BASE}")
except Exception as e:
    print(f"❌ 出力リンクエラー: {e}")

# 8) cloudflared準備
print("8️⃣ cloudflared準備中...")
cf_path = '/usr/local/bin/cloudflared'
if not Path(cf_path).exists():
    safe_run(['wget', '-q',
              'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64',
              '-O', cf_path], check=True)
    os.chmod(cf_path, 0o755)

print("✅ 初期セットアップ完了！次のセル2でForgeを起動してください。")
print("🖼️ 生成される画像は自動的にGoogle Driveに保存されます。")


🚀 Stable Diffusion WebUI Forge - 最終版セットアップ開始...
1️⃣ Google Driveマウント中...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2️⃣ Forgeリポジトリ取得中...
3️⃣ 2025年環境最適化中...
4️⃣ 重要依存関係インストール中...
   📦 gradio>=4.0.0
   📦 fastapi>=0.110.0
   📦 uvicorn>=0.23.0
5️⃣ モデルフォルダリンク中...
  📦 退避: Stable-diffusion → Stable-diffusion.backup
  ✅ Stable-diffusion → Stable-diffusion (0個)
  ✅ Lora → Lora (0個)
  📦 退避: VAE → VAE.backup
  ✅ VAE → VAE (0個)
  ✅ ControlNet → ControlNet (3個)
  ✅ embeddings → embeddings (0個)
  ⚠️ hypernetworks: フォルダなし
6️⃣ モデルリンク完了: 5/6個
7️⃣ 画像出力をGoogle Driveに設定中...
✅ 画像自動保存設定完了: outputs → /content/drive/MyDrive/AI/outputs/forge
8️⃣ cloudflared準備中...
✅ 初期セットアップ完了！次のセル2でForgeを起動してください。
🖼️ 生成される画像は自動的にGoogle Driveに保存されます。


In [ ]:
# 現在のコードの「8️⃣ cloudflared準備中...」の前に追加
# 🎯【追加】拡張機能永続化設定
print("8️⃣ 拡張機能永続化設定中...")
EXTENSIONS_BASE = AI_BASE / 'forge' / 'extensions'
EXTENSIONS_BASE.mkdir(parents=True, exist_ok=True)
forge_ext = WEBUI_DIR / 'extensions'

# 既存のextensionsフォルダを安全に処理
if forge_ext.exists() and not forge_ext.is_symlink():
    if any(forge_ext.iterdir()):
        # 既存の拡張機能をGoogle Driveに移動
        for item in forge_ext.iterdir():
            if item.is_dir():
                dest = EXTENSIONS_BASE / item.name
                if not dest.exists():
                    shutil.move(str(item), str(dest))
                    print(f"  📦 拡張機能移動: {item.name}")
    shutil.rmtree(forge_ext, ignore_errors=True)

# Google DriveへのシンボリックリンクPython作成
try:
    os.symlink(EXTENSIONS_BASE, forge_ext, target_is_directory=True)
    print(f"✅ 拡張機能永続化完了: extensions → {EXTENSIONS_BASE}")

    # おすすめ拡張機能の自動インストール
    print("9️⃣ おすすめ拡張機能自動インストール中...")
    extensions = {
        'adetailer': {
            'url': 'https://github.com/Bing-su/adetailer.git',
            'desc': '顔・手の自動修正（最重要）'
        },
        'sd-dynamic-prompts': {
            'url': 'https://github.com/adieyal/sd-dynamic-prompts.git',
            'desc': 'ランダム生成・創作支援'
        },
        'sd-webui-prompt-all-in-one': {
            'url': 'https://github.com/Physton/sd-webui-prompt-all-in-one.git',
            'desc': 'プロンプト入力支援'
        },
        'stable-diffusion-webui-rembg': {
            'url': 'https://github.com/AUTOMATIC1111/stable-diffusion-webui-rembg.git',
            'desc': '背景除去機能'
        }
    }

    installed_count = 0
    for ext_name, ext_info in extensions.items():
        ext_path = EXTENSIONS_BASE / ext_name
        print(f"📦 {ext_name}: {ext_info['desc']}")

        if ext_path.exists():
            print("  ⚡ 既存")
            installed_count += 1
        else:
            try:
                safe_run(['git', 'clone', '--depth', '1', ext_info['url'], str(ext_path)], timeout=180)
                print("  ✅ インストール完了")
                installed_count += 1
            except:
                print("  ❌ インストール失敗")

    print(f"🎉 拡張機能インストール完了: {installed_count}/{len(extensions)}個")

except Exception as e:
    print(f"❌ 拡張機能リンクエラー: {e}")

# ADetailer検出モデルの永続化（オプション）
print("🔟 ADetailer検出モデル永続化中...")
drive_adetailer = AI_BASE / 'models' / 'adetailer'
drive_adetailer.mkdir(parents=True, exist_ok=True)

adetailer_models = [
    'https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8n.pt',
    'https://huggingface.co/Bingsu/adetailer/resolve/main/face_yolov8s.pt',
    'https://huggingface.co/Bingsu/adetailer/resolve/main/hand_yolov8n.pt'
]

for url in adetailer_models:
    model_name = url.split('/')[-1]
    model_path = drive_adetailer / model_name
    if not model_path.exists():
        try:
            safe_run(['wget', '-q', '-O', str(model_path), url], timeout=120)
            print(f"  ✅ {model_name}")
        except:
            print(f"  ❌ {model_name}")

# Forgeのadetailerフォルダにリンク
forge_adetailer = WEBUI_DIR / 'models' / 'adetailer'
if forge_adetailer.exists() and not forge_adetailer.is_symlink():
    shutil.rmtree(forge_adetailer, ignore_errors=True)
if not forge_adetailer.is_symlink():
    os.symlink(drive_adetailer, forge_adetailer, target_is_directory=True)
    print("✅ ADetailerモデルリンク完了")


In [3]:
# ================================================================
# Stable Diffusion WebUI Forge - 起動（毎回実行用）
# ================================================================
import os, subprocess, time, re, requests
from pathlib import Path

WEBUI_DIR = Path('/content/stable-diffusion-webui-forge')
PORT = 7860
CF_PATH = '/usr/local/bin/cloudflared'

print("🚀 Forge起動開始...")

# 1) 環境確認
if not (WEBUI_DIR / 'launch.py').exists():
    print("❌ Forgeが見つかりません。セル1を実行してください。")
    raise FileNotFoundError("Forge not found")

# 2) 既存プロセス停止
print("1️⃣ プロセス停止中...")
os.system("pkill -f cloudflared >/dev/null 2>&1 || true")
os.system("pkill -f launch.py >/dev/null 2>&1 || true")
os.system("fuser -k 7860/tcp >/dev/null 2>&1 || true")
time.sleep(2)

# 3) 環境変数設定
env = os.environ.copy()
env.update({
    'PYTHONUNBUFFERED': '1',
    'PYTORCH_CUDA_ALLOC_CONF': 'max_split_size_mb:128'
})

# 4) Forge起動
print("2️⃣ Forge起動中...")
forge_process = subprocess.Popen(
    ['python3', 'launch.py', '--server-name', '0.0.0.0', '--port', str(PORT), '--skip-version-check'],
    cwd=str(WEBUI_DIR),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
    env=env
)

print(f"Forge PID: {forge_process.pid}")

# 5) 起動監視（リアルタイム表示）
print("3️⃣ 起動監視中...")
forge_ready = False
start_time = time.time()

for line_num, line in enumerate(forge_process.stdout, 1):
    print(line, end='')

    # 起動成功判定
    if any(keyword in line for keyword in [
        'Running on local URL', 'Uvicorn running', 'Model loaded',
        'Startup time:', 'Loading weights'
    ]):
        forge_ready = True
        print("✅ Forge起動成功！")
        break

    # タイムアウト判定（5分）
    if time.time() - start_time > 300:
        print("⚠️ 起動監視タイムアウト")
        break

    # 出力制限
    if line_num > 150:
        print("📝 ログ出力制限")
        break

# 6) HTTP応答確認
if forge_ready or forge_process.poll() is None:
    print("4️⃣ HTTP応答確認中...")
    time.sleep(3)

    http_ok = False
    for _ in range(15):  # 45秒確認
        try:
            response = requests.get(f'http://127.0.0.1:{PORT}', timeout=3)
            if response.status_code == 200:
                http_ok = True
                print("✅ HTTP応答OK")
                break
        except:
            pass
        time.sleep(3)

    if http_ok:
        # 7) 外部アクセス用トンネル作成
        print("5️⃣ 外部アクセス用トンネル作成中...")
        tunnel_process = subprocess.Popen(
            [CF_PATH, 'tunnel', '--url', f'http://127.0.0.1:{PORT}', '--no-autoupdate'],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        # 8) URL抽出・表示
        print("6️⃣ アクセスURL取得中...")
        for line in tunnel_process.stdout:
            print(line, end='')
            url_match = re.search(r'https://[^\s]*trycloudflare\.com[^\s]*', line)
            if url_match:
                access_url = url_match.group(0)
                print(f"\n🎉 Forge アクセスURL: {access_url}")
                print("✅ 上記URLをクリックしてForgeをご利用ください！")
                print("💡 WebUIで🔄 Refreshボタンを押してモデルを確認")
                print("🖼️ 生成画像は自動的にGoogle Driveに保存されます")
                break
    else:
        print("❌ HTTP応答なし")
else:
    print("❌ Forge起動失敗")

print("\n📝 使用方法:")
print("  1. 上記URLにアクセス")
print("  2. 'Stable Diffusion checkpoint'でモデル選択")
print("  3. プロンプト入力して'Generate'で画像生成")
print("  4. 生成画像は MyDrive/AI/outputs/forge に自動保存")


🚀 Forge起動開始...
1️⃣ プロセス停止中...
2️⃣ Forge起動中...
Forge PID: 2775
3️⃣ 起動監視中...
INCOMPATIBLE PYTHON VERSION

This program is tested with 3.10.6 Python, but you have 3.12.12.
If you encounter an error with "RuntimeError: Couldn't install torch." message,
or any other error regarding unsuccessful package (library) installation,
please downgrade (or upgrade) to the latest version of 3.10 Python
and delete current Python and "venv" folder in WebUI's directory.

You can download 3.10 Python from here: https://www.python.org/downloads/release/python-3106/



Use --skip-python-version-check to suppress this warning.
fatal: No names found, cannot describe anything.
Python 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Version: f2.0.1v1.10.1-1.10.1
Commit hash: dfdcbab685e57677014f05a3309b48cc87383167
Installing clip
Installing open_clip
Cloning assets into /content/stable-diffusion-webui-forge/repositories/stable-diffusion-webui-assets...
Cloning into '/content/stable-diffusion-webui-forge/repo